# Lab 7
+ ## Автор: Роман Кривохижа
+ ## Група: ІС-72

****
****
****

## Module importing

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from IPython.display import display

In [4]:
import time 

In [5]:
sns.set_style('darkgrid')
%matplotlib inline

## Algorithm implementation

In [6]:
class Rand:
    """
    Генерація випадкового числа за заданим законом розподілу
    """
    @staticmethod
    def exp(time_mean):
        a = 0
        while a == 0:
            a = np.random.rand()
        return -time_mean * np.log(a)
    
    @staticmethod
    def unif(time_min, time_max):
        a = 0
        while a == 0:
            a = np.random.rand()
        a = time_min + a * (time_max - time_min)
        return a
    
    @staticmethod
    def norm(time_mean, time_deviation):
        return np.random.normal(loc=time_mean, scale=time_deviation)

In [7]:
class Position:
    def __init__(self, name, num_of_markers=0, description=None):
        self.num_of_markers = num_of_markers
        self.name = name
        self.description = description
        self.type = 'Position'
        
    def add_markers(self, amount_of_added_markers):
        self.num_of_markers += amount_of_added_markers
        
    def remove_markers(self, amount_of_removed_markers):
        self.num_of_markers -= amount_of_removed_markers
        
    def __repr__(self):
        return f'Name: {self.name}' + ("\nDescription: " + self.description if self.description else "-")

In [8]:
class Transition:
    def __init__(self, name, priority=0, probability=None, distribution=None, delay_params=None, description=None):
        self.name = name
        self.priority = priority
        self.probability = probability
        self.distribution = distribution
        self.delay_params = delay_params
        self.description = description
        self.type = 'Transition'
        
        self.input_arcs = []
        self.output_arcs = []
        
    def is_available(self):
        return all(arc.is_available() for arc in self.input_arcs)
    
    def make_a_transition(self):
        for arc in self.input_arcs:
            arc.move_from()
        
        if self.distribution is not None: self.distribution(*self.delay_params)
        
        for arc in self.output_arcs:
            arc.move_to()
            
    def __repr__(self):
        return f'Name: {self.name}' + ("\nDescription: " + self.description if self.description else "-")

In [9]:
class Arc:
    def __init__(self, start, end, multiplicity=1):
        self.start = start
        self.end = end
        self.multiplicity = multiplicity
        self.type = 'Arc'
        
    def is_available(self):
        return self.start.num_of_markers >= self.multiplicity
    
    def move_from(self):
        self.start.remove_markers(self.multiplicity)
        
    def move_to(self):
        self.end.add_markers(self.multiplicity)
        
    def __repr__(self):
        return f'{self.start.name}({self.start.type}) ---> {self.end.name}({self.end.type})'

In [10]:
class Model:
    def __init__(self, transitions, positions, arcs):
        self.transitions = transitions
        self.positions = positions
        self.arcs = arcs
        self.tcurr = 0
        
        diff_stats = {'avg_markers': 0, 'max_markers': 0, 'min_markers': np.inf, 'result_markers': 0, 'n_cnt': 0}
        self.positions_stats = {position.name: diff_stats.copy() for position in self.positions}
        
    def simulate(self, time_modeling):
        self.time_modeling = time_modeling
        
        while self.tcurr < self.time_modeling:
            we_have_available_transition = any(transition.is_available() for transition in self.transitions)
            
            if not we_have_available_transition:
                break
                
            resolved_transitions = self.resolve_conflict()
            
            self.do_calc()
            for transition in resolved_transitions:
                if transition.is_available():
                    transition.make_a_transition()
            self.tcurr += 1
            
        self.print_results()
            
    def resolve_conflict(self):
        resolved_list = []
        conflict_list = []
        
        # розділяємо нормальні стани та конфліктні
        for position in self.positions:
            output_arcs_for_position = [*filter(lambda arc: arc.start is position, self.arcs)]
            resolved = [*filter(lambda trans: trans.is_available(),
                                map(lambda arc: arc.end, output_arcs_for_position))
                       ]
            if len(resolved) > 1:
                conflict_list.append(resolved)
            else:
                resolved_list += resolved
        
        # вирішуємо конфлікти
        for conflict in conflict_list:
            # конфлікт за ймовірністю
            if conflict[0].probability is not None:
                p = [c.probability for c in conflict]
                resolved = np.random.choice(conflict, p=p)
            # конфлікт за пріоритетом
            elif len(np.unique([c.priority for c in conflict if c.priority is not None])) > 1:
                priority = [c.priority for c in conflict]
                resolved = conflict[np.argmax(priority)]
            else:
                resolved = np.random.choice(conflict)
                
            resolved_list += [resolved]
            
        return resolved_list
    
    def do_calc(self):
        for position in self.positions:
            self.positions_stats[position.name]['avg_markers'] += position.num_of_markers
            self.positions_stats[position.name]['max_markers'] = max(self.positions_stats[position.name]['max_markers'], position.num_of_markers)
            self.positions_stats[position.name]['min_markers'] = min(self.positions_stats[position.name]['min_markers'], position.num_of_markers)
            self.positions_stats[position.name]['result_markers'] = position.num_of_markers
            self.positions_stats[position.name]['n_cnt'] += 1
        
    def prepare_stats(self):
        for position in self.positions:
            self.positions_stats[position.name]['avg_markers'] /= self.positions_stats[position.name]['n_cnt']
            del self.positions_stats[position.name]['n_cnt']
    
    def print_results(self):
        self.prepare_stats()
        
        rows = []
        for position in self.positions:
            self.positions_stats[position.name]['Name'] = position.name
            self.positions_stats[position.name]['Description'] = position.description if position.description else "-"
            
            rows.append(self.positions_stats[position.name])
            
        df = pd.DataFrame(rows, columns=list(self.positions_stats[position.name].keys()))
        display(df)

## Model verification

<img src="img/petri_test.jpg">

In [11]:
class SimpleModel:
    def __init__(self):
        # Задамо позиції
        p1 = Position('P1', num_of_markers=1, description='Надходження деталей 1-го типу')
        p2 = Position('P2', num_of_markers=1, description='Надходження деталей 2-го типу')
        p3 = Position('P3', num_of_markers=0, description='Черга деталей 1-го типу')
        p4 = Position('P4', num_of_markers=0, description='Черга деталей 2-го типу')
        p5 = Position('P5', num_of_markers=0, description='Кількість заповнених секцій')
        p6 = Position('P6', num_of_markers=1, description='Надходження секції конвеєра')
        p7 = Position('P7', num_of_markers=0, description=None)
        p8 = Position('P8', num_of_markers=0, description='Кількість порожніх секцій')
        
        # Задамо переходи
        t1 = Transition('T1', description='Надходження деталі 1-го типу')
        t2 = Transition('T2', description='Надходження деталі 2-го типу')
        t3 = Transition('T3', priority=10, description='Комплектація')
        t4 = Transition('T4', description='Надходження секції конвеєра')
        t5 = Transition('T5', description='Порожня секція')
        
        # Задамо дуги
        a_p1_t1 = Arc(start=p1, end=t1)
        a_t1_p1 = Arc(start=t1, end=p1)
        a_t1_p3 = Arc(start=t1, end=p3, multiplicity=10)
        a_p3_t3 = Arc(start=p3, end=t3, multiplicity=20)
        
        a_p2_t2 = Arc(start=p2, end=t2)
        a_t2_p2 = Arc(start=t2, end=p2)
        a_t2_p4 = Arc(start=t2, end=p4, multiplicity=10)
        a_p4_t3 = Arc(start=p4, end=t3, multiplicity=20)
        
        a_t3_p5 = Arc(start=t3, end=p5)
        
        a_p6_t4 = Arc(start=p6, end=t4)
        a_t4_p6 = Arc(start=t4, end=p6)
        a_t4_p7 = Arc(start=t4, end=p7)
        
        a_p7_t3 = Arc(start=p7, end=t3)
        a_p7_t5 = Arc(start=p7, end=t5)
        
        a_t5_p8 = Arc(start=t5, end=p8)
        
        # Задамо вхідні дуги для кожної похиції
        t1.input_arcs = [a_p1_t1]
        t2.input_arcs = [a_p2_t2]
        t3.input_arcs = [a_p3_t3, a_p4_t3, a_p7_t3]
        t4.input_arcs = [a_p6_t4]
        t5.input_arcs = [a_p7_t5]
        
        # Задамо вихідні дуги для кожної позиції
        t1.output_arcs = [a_t1_p1, a_t1_p3]
        t2.output_arcs = [a_t2_p2, a_t2_p4]
        t3.output_arcs = [a_t3_p5]
        t4.output_arcs = [a_t4_p6, a_t4_p7]
        t5.output_arcs = [a_t5_p8]
        
        # Створимо мережу Петрі
        petri_network = Model(positions=[p1,p2,p3,p4,p5,p6,p7,p8],
                              transitions=[t1,t2,t3,t4,t5],
                              arcs=[a_p1_t1,a_t1_p1,a_t1_p3,a_p3_t3,a_p2_t2,a_t2_p2,
                                    a_t2_p4,a_p4_t3,a_t3_p5,a_p6_t4,a_t4_p6,a_t4_p7,
                                    a_p7_t3,a_p7_t5,a_t5_p8])
        petri_network.simulate(100)

In [12]:
_ = SimpleModel()

,avg_markers,max_markers,min_markers,result_markers,Name,Description
0,1.00,1,1,1,P1,Надходження деталей 1-го типу
1,1.00,1,1,1,P2,Надходження деталей 2-го типу
2,14.80,20,0,10,P3,Черга деталей 1-го типу
3,14.80,20,0,10,P4,Черга деталей 2-го типу
4,24.01,49,0,49,P5,Кількість заповнених секцій
5,1.00,1,1,1,P6,Надходження секції конвеєра
6,0.99,1,0,1,P7,-
7,24.50,49,0,49,P8,Кількість порожніх секцій
